# Lecture 5: Simulations and Uncertainty

#### Zhentao Shi



<img src="graph/macau.jpg" width="1000">

In [ ]:
import numpy as np
import pandas as pd
import random

## Simulation


* check finite-sample performance
* bootstrap, a data-driven inference procedure
* generate non-standard distributions
* approximate integrals with no analytic expressions

## Appetizer

calculating $\pi$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.add_patch(plt.Rectangle((-1,-1), 2, 2, fill=False))
ax.add_artist(plt.Circle((0, 0), 1, fill=False))
x = np.random.uniform(-1, 1, size=100)
y = np.random.uniform(-1, 1, size=100)
ax.scatter(x, y, s=25, c='blue', marker='o')
plt.show()


By laws of large numbers, $\pi$ can be approximated by a stochastic algorithm

$$
E\left[\boldsymbol{1}\left\{ x^{2}+y^{2}\leq1\right\} \right] = \frac{\pi r^{2}}{\left(2r\right)^{2}}= \frac{\pi}{4}
$$

it implies  $\pi=4\times E[ 1 \{  x^{2}+y^{2}\leq1 \}]$. 

In [ ]:
import numpy as np

n = 20000000

# set a random seed
np.random.seed(20250221)

Z = 2 * np.random.rand(n, 2) - 1 # uniform distribution in [-1, 1]

inside = np.mean(np.sqrt(np.sum(Z**2, axis=1)) <= 1)
print(f"The estimated pi = {inside * 4:.6f}")


* Sample size can be made as large as the computer's memory permits.
* Iterate it with average of averages and so on, for higher accuracy.

## Handwriting Scripts

This course promotes vibe coding.

Human wrote code by hand, before AI.

* A script is a piece of code for a particular purpose. 
* Thousands of lines are not written from the beginning to the end. 

* Recursive development
  * small manageable tasks
  * test code constantly 
  * encapsulate into DIY functions

* Integrate small pieces into the super structure. 
* Add comments to the script to facilitate readability
* Scprit (XXX.py) is more suitable than Jupyter notebook

## Finite Sample Evaluation


* Real world sample is finite
* Asymptotic theory is a mathematical apparatus to approximate finite sample distributions
* Modern econometric theory is built on asymptotics
* Simulation is one way to evaluate the accuracy of approximation.

## Example

For the OLS estimator, 

* Classical views $X$ as fixed regressions and only cares about the randomness of the error term.
* Modern econometrics textbook emphasizes that a random $X$ is more appropriate
for econometrics applications. 
* In rigorous textbooks, the moment of $X$ is explicitly stated as $E[X_i X_i'] < \infty$.

* A Pareto distribution with shape coefficient between 1 and 2 has finite population mean, but infinite variance. 
* If $X$ follows a
[Pareto distribution](https://en.wikipedia.org/wiki/Pareto_distribution) with shape coefficient 1.5, it violates the assumptions for OLS stated in most of econometric textbooks.
* Question: Is asymptotic inferential theory for the OLS estimator valid? 

We write a script to investigate this problem. The following steps develop the code.

 1. given a sample size, get the OLS `b_hat` and its associated `t_value`.
 2. wrap `t_value` as a user-defined function so that we can reuse it for many times.
 3. given a sample size, report the size under two distributions.
 4. wrap step 3 again as a user-defined function, ready for different sample sizes.
 5. develop the super structure to connect the workhorse functions.
 6. add comments and documentation.


### $t$-statistic

$$
\sqrt{n} (\hat{\beta} - \beta_0) |X = (X'X/n)^{-1}  (X' e /\sqrt{n}),
$$

the $k$-th element of the vector coefficient conditional on $X$ is
$$
\widehat{\beta}_{k}|X=\eta_{k}'\widehat{\beta}|X
\sim N\left(\beta_{k},\sigma^{2}\left(X'X\right)_{kk}^{-1}\right).
$$

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t

def simulation(n, dist='normal', df=5):
    """
    Simulate the t-value of the second parameter in a linear regression model
    with n samples and the specified distribution of errors.
    """
    
    if dist == 'normal':
        e = np.random.normal(size=n)
    elif dist == 't':
        e = np.random.standard_t(df, size=n)

    b0 = np.array([1, 2])
    X = np.column_stack((np.ones(n), np.random.pareto(1.5, size=n)))
    # the above X is generated from a Pareto distribution
    Y = X.dot(b0) + e

    bhat = np.linalg.inv(X.T @ X) @ (X.T @ Y)
    bhat2 = bhat[1] # parameter we want to test

    e_hat = Y - X.dot(bhat)
    sigma_hat_square = np.sum(e_hat**2) / (n - 2)
    sig_B = np.linalg.inv(X.T.dot(X)) * sigma_hat_square
    t_value_2 = (bhat2 - b0[1]) / np.sqrt(sig_B[1, 1])

    return t_value_2



In [ ]:
from scipy.stats import t, norm

def report(n, Rep):
    # collect the test size from the two distributions
    # this function contains some repetitive code, but is OK for such a simple one
    TEST_SIZE = np.zeros(3)

    # e ~ normal distribution, under which the t-dist is exact
    # generate the t-statistics from the normal distribution
    Res = np.array([simulation(n, 'normal') for i in range(Rep)])
    
    TEST_SIZE[0] = np.mean(np.abs(Res) > t.ppf(0.975, n-2))
    TEST_SIZE[1] = np.mean(np.abs(Res) > norm.ppf(0.975))

    # e ~ t-distribution, under which the exact distribution is complicated.
    # we rely on asymptotic normal distribution for inference instead
    Res = np.array([simulation(n, 't', df) for i in range(Rep)])
    TEST_SIZE[2] = np.mean(np.abs(Res) > norm.ppf(0.975))

    return TEST_SIZE


In [ ]:
Rep = 1000
df = 1  # t dist. with df = 1 is Cauchy

# run the calculation of the empirical sizes for different sample sizes
NN = [5, 10, 200, 2000, 20000]

RES = pd.DataFrame(np.zeros((len(NN), 4)), columns=['n', 'exact', 'normal.asym', 'cauchy.asym'])

for i, n in enumerate(NN): # loop over different sample sizes
    # enumerate is a built-in Python function that returns an iterator containing the index and value of a list

    RES.iloc[i, 0] = n
    RES.iloc[i, 1:] = report(n, Rep)

print(RES)


### Simulation Results

* 1st column: the error is normal, use exact distribution theory to find the critical value (according to $t$-distribution.)

* 2nd column: still uses the normal error
  * change the critical value to asymptotic normal distribution
  
* 3rd column: error distribution is Cauchy
  * asymptotic approximation breaks down
  * test size does not converge to the nominal 5%  

### Observations

* $X$ is always Pareto 
* distribution of $X$ doesn't matter in our simulations

### Justification

* Self-normalized $t$ statistic does not break down despite that $X'X/n$ does not converge
* Regardless the distribution of $X$, when the error term is normal 
  * numerator follows $N(0,1)$
  * demonimator follows $\chi^2$

## AI Agent

to be implemented in SCRP.

* **Prompt**: Create a demo of using simulations to evaluate the value-at-risk of a portfolio of 5 assets. Try normal distributions and heavy-tail distributions. Maintain the correlation between the assets. 

## Application of Simulation: Portfolio VaR 

(from vibe coding)

We simulate a portfolio of 5 assets and compare Value-at-Risk (VaR) under:

1. A multivariate normal return model.
2. A multivariate Student-$t$ model (heavy tails).

Both models use the same target correlation matrix so cross-asset dependence is preserved.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(5821)

# Portfolio setup
n_assets = 5
n_sims = 200000
weights = np.array([0.25, 0.20, 0.20, 0.20, 0.15])

# Daily expected returns (example values)
mu = np.array([0.0005, 0.0004, 0.00045, 0.00035, 0.0003])

# Daily volatilities
vol = np.array([0.012, 0.010, 0.011, 0.009, 0.013])

# Correlation matrix (must be symmetric positive definite)
corr = np.array([
    [1.00, 0.55, 0.40, 0.30, 0.35],
    [0.55, 1.00, 0.45, 0.25, 0.30],
    [0.40, 0.45, 1.00, 0.50, 0.20],
    [0.30, 0.25, 0.50, 1.00, 0.15],
    [0.35, 0.30, 0.20, 0.15, 1.00],
])

# Covariance used by the Gaussian model
cov = np.outer(vol, vol) * corr

# Utility: VaR and ES from simulated portfolio returns
def var_es(returns, alpha=0.99):
    # Loss = -return. VaR_{alpha} is alpha-quantile of loss.
    losses = -returns
    var = np.quantile(losses, alpha)
    es = losses[losses >= var].mean()
    return var, es


### 1) Correlated Normal Simulation

In [ ]:
# Multivariate normal asset returns
r_norm = rng.multivariate_normal(mean=mu, cov=cov, size=n_sims)
port_r_norm = r_norm @ weights

# Check that simulated cross-asset correlation matches target
corr_norm_sim = np.corrcoef(r_norm, rowvar=False)
np.set_printoptions(precision=3, suppress=True)
print("Target correlation matrix:", corr)
print("Simulated correlation (normal):", corr_norm_sim)


### 2) Correlated Heavy-Tail Simulation via Left-Tail-Dependent Copula

We use a **Clayton copula** to create strong lower-tail dependence:

- Clayton parameter `theta > 0` controls lower-tail dependence.
- Pairwise lower-tail dependence is

$$
\lambda_L = 2^{-1/	\theta}.
$$

Then we map copula uniforms to Student-`t` marginals with `t.ppf`.

Note: This simple Clayton construction is exchangeable (same dependence strength across pairs).

In [ ]:
from scipy.stats import t

nu = 3
# Clayton copula parameter: positive => lower-tail dependence.
# theta=0.6 gives noticeable left-tail dependence while keeping average
# linear correlation closer to the target matrix than very large theta.
theta = 0.8
lambda_L = 2 ** (-1 / theta)
print(f"Theoretical lower-tail dependence lambda_L = {lambda_L:.3f}")

# Sample Clayton copula uniforms (Marshall-Olkin representation)
w = rng.gamma(shape=1 / theta, scale=1.0, size=n_sims)
e = rng.exponential(scale=1.0, size=(n_sims, n_assets))
u_cop = (1 + e / w[:, None]) ** (-1 / theta)

# Avoid exact 0/1 before inverse CDF transform
eps = 1e-12
u_cop = np.clip(u_cop, eps, 1 - eps)

# Heavy-tail marginals via inverse Student-t CDF
x_t = t.ppf(u_cop, df=nu)

# Standardize to unit variance (nu > 2), then scale to target vols
x_t = x_t * np.sqrt((nu - 2) / nu)
r_t = mu + x_t * vol
port_r_t = r_t @ weights

corr_t_sim = np.corrcoef(r_t, rowvar=False)
spear_t_sim = pd.DataFrame(r_t).corr(method="spearman").to_numpy()

# Compare average off-diagonal correlation to target matrix
def avg_offdiag(a):
    n = a.shape[0]
    return (a.sum() - np.trace(a)) / (n * (n - 1))

avg_target_corr = avg_offdiag(corr)
avg_sim_corr = avg_offdiag(corr_t_sim)

# Empirical lower-tail dependence for one pair (assets 1 and 2)
q = 0.05
lambda_L_hat = ((u_cop[:, 0] <= q) & (u_cop[:, 1] <= q)).mean() / q

np.set_printoptions(precision=3, suppress=True)
print("Target correlation matrix:", corr)
print("Simulated Pearson correlation (Clayton-copula t):", corr_t_sim)
print("Simulated Spearman correlation (Clayton-copula t):", spear_t_sim)
print(f"Average off-diagonal corr: target={avg_target_corr:.3f}, simulated={avg_sim_corr:.3f}")
print(f"Empirical lower-tail dependence (q={q:.2f}, assets 1&2): {lambda_L_hat:.3f}")


### 3) VaR / ES Comparison

In [ ]:
alphas = [0.95, 0.99]
rows = []
for a in alphas:
    var_n, es_n = var_es(port_r_norm, alpha=a)
    var_t, es_t = var_es(port_r_t, alpha=a)
    rows.append([f"{int(a*100)}%", "Normal", var_n, es_n])
    rows.append([f"{int(a*100)}%", f"Student-t (nu={nu})", var_t, es_t])

res = pd.DataFrame(rows, columns=["Confidence", "Model", "VaR", "ES"])
res


In [ ]:
# Visual comparison of portfolio return tails
plt.figure(figsize=(8, 4))
plt.hist(port_r_norm, bins=200, density=True, alpha=0.55, label="Normal")
plt.hist(port_r_t, bins=200, density=True, alpha=0.45, label=f"Student-t (nu={nu})")
plt.xlim(-0.08, 0.04)
plt.title("Portfolio Return Distribution: Normal vs Heavy-Tail")
plt.xlabel("Daily portfolio return")
plt.ylabel("Density")
plt.legend()
plt.show()


### Interpretation

- The Clayton-copula model injects strong **left-tail dependence** (joint crashes are more likely).
- Combined with Student-`t` marginals, this typically raises 99% VaR and ES versus the normal model.
- This setup is useful when downside co-movement risk is a primary concern.

## World View about Uncertainty


* Fundamental question: how to quantify uncertainty.
  * data $(X_1,X_2,\ldots,X_n)$
  * sample mean $\bar{X}$
  * sample variance $s$
  * frequentist confidence interval about the population mean

* Asymptotics is imaginary.
* Let's be realistic: we have a finite sample $n$.

## Bootstrap

* Let $X_1, X_2, \ldots, X_n \sim F$ be an i.i.d. sample of $n$ observations following a distribution $F$. 
* The finite sample distribution of a statistic $T_n(\theta)\sim G_n(\cdot, F)$ usually depends on the sample size $n$, as well as the unknown true distribution $F$. 


### Key Idea

* Bootstrap replaces the unknown distribution $F$ in $G_n(\cdot, F)$ by the empirical distribution function

$$
\hat{F}_n(\cdot) = n^{-1} \sum_{i=1}^n 1\{\cdot \leq X_i\}
$$

* Bootstrap inference is drawn from the bootstrap distribution

$$
G_n(\cdot, \hat{F}_n)
$$


### Compare to Asymptotic Theory

* Bootstrap is a finite-sample practice 
* It doesn't refer to an imaginary world where $n\to \infty$ at its face value


* Asymptotic theory approximates $G_n(\cdot, F)$ by its limit 

$$
G(\cdot, F) := \lim_{n\to\infty} G_n(\cdot, F).
$$ 

* In many cases $G(\cdot, F)$ is independent of $F$ and it becomes $G(\cdot)$. Such a $T_n(\theta)$ is called *asymptotically pivotal*, 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF

x = np.random.beta(2,2,size=10)
ecdf = ECDF(x)

# get unique sorted values of x and corresponding values of y
ux, uy = np.unique(ecdf.x, return_index=False, return_inverse=False, return_counts=False), ecdf.y[np.unique(ecdf.y, return_index=True, return_inverse=False, return_counts=False)[1]]

# add last point (1,1) to the step function
ux = np.concatenate((ux, [1]))
uy = np.concatenate((uy, [1]))

# create step function with horizontal lines
plt.plot(ux[:-1], uy[:-1], drawstyle='steps-pre')
plt.plot([ux[-2], ux[-1]], [uy[-2], uy[-2]], color='C0', linestyle='--')
plt.plot([ux[-1], ux[-1]], [uy[-2], uy[-1]], color='C0', linestyle='--')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('ECDF for uniform distribution')
plt.show()

### Nonparametric Bootstrap

* Implementation of bootstrap is a simulation exercise. 
* In an i.i.d. environment, $n$ observations are drawn with equal weight and **with replacement** from the realized sample

### Variants of Bootstrap Schemes

* Block bootstrap: preserve dependence structure
  * dependent dataset such as time series
  * clustering data or networks

* parametric bootstrap
  * In regressions we fix the regressors $X$

In [ ]:
import pandas as pd
import numpy as np
n = 9  # real sample size
d0 = pd.DataFrame({'x': np.random.normal(size=n)})
print(d0)

# bootstrap
boot_Rep = 3  # bootstrap 3 times
d_boot = np.zeros((n, boot_Rep))  # save the bootstrap sample
d_boot_index = np.zeros((n, boot_Rep), dtype=int)  # save the bootstrap sample index
for b in range(boot_Rep):
    boot_index = np.random.choice(d0.index, n, replace=True) # sampling with replacement
    d_boot[:, b] = d0['x'].iloc[boot_index]
    d_boot_index[:, b] = boot_index

d_boot_df = pd.DataFrame(d_boot)  # convert the bootstrap samples to DataFrame
print(d_boot_df)
print(d_boot_index)


**Example**: In one sample, there is only one sample mean. How to compute the variance of the sample mean? 

In [ ]:
np.var(d0['x'])/n # the variance of the sample mean

What if you forget the formula?

### Bootstrap Estimation

* Bootstrap is simply a loop.
* Bootstrap is convenient. 
  * Analytic formula of the variance of an econometric estimator can be complex to derive or code up.
  * e.g.: MLE, in particular misspecified MLE (Hessian, Fisher...)

In [ ]:
boot_Rep = 10000  # bootstrap many times times
d_boot = np.zeros((n, boot_Rep))  # save the bootstrap sample 
for b in range(boot_Rep):
    boot_index = np.random.choice(n, n, replace=True)
    d_boot[:, b] = d0['x'][boot_index]
 
np.mean(d_boot, axis=0).var()  # standard deviation of the bootstrap means across boot samples

### Bootstrap Test

* Bootstrap is particularly helpful in inference

### Example

* Test a hypothesis about the population mean. 

* Use $t$-statistic
* Distribution of the sample is either
  * normal
  * zero-centered chi-square 

* We will show that the bootstrap test size is
more precise than that of the asymptotic approximation.

In [ ]:
import numpy as np

def T_stat(Y, mu):
    n = len(Y)
    return np.sqrt(n) * (np.mean(Y) - mu) / np.std(Y, ddof=1)

def boot_test(Y, alpha, boot_Rep):
    n = len(Y)
    boot_T = []

    for r in range(boot_Rep):
        
        resampled_Y = Y[np.random.choice(n, n, replace=True)]
        boot_T.append(abs(T_stat(resampled_Y, np.mean(Y))))
        # this is a key line
        # `mu` must be replaced as mean(Y)

    boot_critical_value = np.quantile(boot_T, 1 - alpha)
    return boot_critical_value


In [ ]:
Y = np.random.normal(size = 20)
T_stat(Y, 3)

In [ ]:
boot_test(Y, 0.05, 1000) # bootstrap critical value

## Bootstrap Application: Market Risk for an Equity Trading Desk

A risk team often reports **1-day Value-at-Risk (VaR)** and **Expected Shortfall (ES)** to management.
Because tail risk is hard to estimate analytically, bootstrap is a practical way to quantify uncertainty in these risk metrics.

In this example, we create a stylized daily return series for a $50 million equity portfolio and use bootstrap to build confidence intervals for 99% VaR and ES.

In this exercise, there is only one sample of n observations. There is no repeated samples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(5821)

# Stylized daily returns: heavy tails + changing volatility (industry-like behavior)
n_days = 252 * 3  # about 3 years of trading days
portfolio_value = 50_000_000

vol_regime = np.random.choice([0.008, 0.015, 0.025], size=n_days, p=[0.6, 0.3, 0.1])
shock = np.random.standard_t(df=5, size=n_days) / np.sqrt(5 / (5 - 2))
# when the true distribution is a "t", the tail prob is difficult to analyze 

daily_returns = 0.0002 + vol_regime * shock

daily_pnl = portfolio_value * daily_returns

print(f"Sample size: {n_days} trading days")
print(f"Average daily return: {daily_returns.mean():.4%}")
print(f"Daily volatility: {daily_returns.std():.4%}")


In [ ]:
# Point estimates from the observed sample
alpha = 0.01  # 99% VaR
var_99 = -np.quantile(daily_pnl, alpha)
es_99 = -daily_pnl[daily_pnl <= np.quantile(daily_pnl, alpha)].mean()

# Bootstrap uncertainty for VaR and ES
B = 5000
boot_var = np.empty(B)
boot_es = np.empty(B)

n = len(daily_pnl)
for b in range(B):
    sample = np.random.choice(daily_pnl, size=n, replace=True)
    q = np.quantile(sample, alpha)
    boot_var[b] = -q
    boot_es[b] = -sample[sample <= q].mean()

# 95% percentile bootstrap confidence intervals
var_ci = np.quantile(boot_var, [0.025, 0.975])
es_ci = np.quantile(boot_es, [0.025, 0.975])

print("99% 1-day risk estimates (USD):")
print(f"VaR point estimate: ${var_99:,.0f}")
print(f"VaR 95% CI:       [${var_ci[0]:,.0f}, ${var_ci[1]:,.0f}]")
print(f"ES point estimate: ${es_99:,.0f}")
print(f"ES 95% CI:        [${es_ci[0]:,.0f}, ${es_ci[1]:,.0f}]")


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].hist(boot_var, bins=40, color="#1f77b4", alpha=0.8)
ax[0].axvline(var_99, color="black", linestyle="--", label="Point estimate")
ax[0].set_title("Bootstrap Distribution of 99% VaR")
ax[0].set_xlabel("VaR (USD)")
ax[0].legend()

ax[1].hist(boot_es, bins=40, color="#d62728", alpha=0.8)
ax[1].axvline(es_99, color="black", linestyle="--", label="Point estimate")
ax[1].set_title("Bootstrap Distribution of 99% ES")
ax[1].set_xlabel("ES (USD)")
ax[1].legend()

plt.tight_layout()
plt.show()


### Interpretation for industry practice

- VaR gives a threshold loss at a chosen confidence level (here, 99%).
- ES captures the *average* loss in the worst 1% tail and is typically more tail-sensitive than VaR.
- Bootstrap confidence intervals communicate estimation uncertainty to risk committees and can support capital or limit-setting discussions.
